In [2]:
from datetime import datetime
import sys
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, BitsAndBytesConfig

In [3]:
prompt_llama2 = """### Instruction:
Use the Task below and the Input given to write the Response, which is a code refactoring Task for specific languages:

### Task:
Change the style of the following SQL statement from the Hive 2x style to the Hive 3x style.

### Input:
%s

### Response:
"""

In [4]:
prompt_codellama = f'''[INST] <<SYS>>Cloudera code refactoring<</SYS>>

Change the style of the following SQL statement from the Hive 2x style to the Hive 3x style.

Hive 2x Style: %s [/INST]'''

In [5]:
prompt_mistral_7b = f'''Cloudera code refactoring [INST] Change the style of the following SQL statement with the Hive 2x style to the Hive 3x style.
Hive 2x Style: %s
[/INST]
Hive 3x Style:'''

In [6]:
BASE_MODEL = 'MISTRAL_7B'  # CODE_LLAMA, LLAMA_2, MISTRAL_7B

if BASE_MODEL == 'LLAMA_2':
    model_path = '/opt/models/Llama-2-7b-chat-hf'
    peft_model_path = '/root/gen-ai/sql-refactor/adapter4llama2/checkpoint-50'
    prompt_format = prompt_llama2
elif BASE_MODEL == 'CODE_LLAMA':
    model_path = '/opt/models/CodeLlama-7b-Instruct-hf'
    peft_model_path = '/root/gen-ai/sql-refactor-codellama/checkpoint-50'
    prompt_format = prompt_codellama
elif BASE_MODEL == 'MISTRAL_7B':
    model_path = '/home/cdsw/models/Mistral-7B-Instruct-v0.2'
    peft_model_path = '/root/gen-ai/sql-refactor/code-refactor-3/checkpoint-8'
    prompt_format = prompt_mistral_7b
else:
    sys.exit(1)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, model_max_length=8192)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    use_cache=True, # False when training
    # do_sample=True,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [ ]:
# model.load_adapter(peft_model_path)
# model = torch.compile(model)

In [ ]:
sql = 'SELECT greatest(col1, col2) from `db_name.abc`;'
# sql = '''CREATE EXTERNAL TABLE `cloudera.test` (value string)
# stored as parquet
# LOCATION '/warehouse/tablespace/managed/hive/cloudera.db/test';'''
# sql = '''CREATE external TABLE `db.table` (field1 string, field2 string)
# location '/warehouse/tablespace/external/hive/example';'''

prompt = prompt_format % sql
print(prompt)

In [30]:
query = 'Does Cloudera CDP Base 7.1.7 support the REPL command?'
paragraph = 'If you want to use REPL commands to replicate Hive ACID tables between CDP Private Cloud Base clusters, ensure that your source cluster is on CDP Private Cloud Base 7.1.8 or a higher version.'

In [23]:
query = "What is Russell Stokes's occupation?"
paragraph = '''Kathleen Cody (actress)  Kathleen Cody (born October 30, 1954), often credited as Kathy Cody, is an American actress. She is best known for her role as the characters Hallie Stokes and Carrie Stokes, on the television series "Dark Shadows", appearing from June 1970 through April 1971. Her career in film and television lasted over 30 years.'''

In [31]:
prompt = '[TEXT] %s\nPlease answer the following question according to the [TEXT] above. Please do not say "according to the text above" or something "is not mentioned in the text above": %s' % (paragraph, query)
print(prompt)

[TEXT] If you want to use REPL commands to replicate Hive ACID tables between CDP Private Cloud Base clusters, ensure that your source cluster is on CDP Private Cloud Base 7.1.8 or a higher version.
Please answer the following question according to the [TEXT] above. Please do not say "according to the text above" or something "is not mentioned in the text above": Does Cloudera CDP Base 7.1.7 support the REPL command?


In [21]:
prompt_template = 'You are a helpful bot who reads texts and answers questions about them. If the text does not match the question, just summary the text. And also do not say something like "according to the text above" or "something is not mentioned in the text". [TEXT] %s [QUESTION] %s [ANSWER] '
prompt = prompt_template % (paragraph, query)
# messages = [
#     {"role": "assistant", "content": ""},
#     {"role": "user", "content": prompt},
# ]
# prompt = tokenizer.apply_chat_template(messages)
prompt

'You are a helpful bot who reads texts and answers questions about them. If the text does not match the question, just summary the text. And also do not say something like "according to the text above" or "something is not mentioned in the text". [TEXT] If you want to use REPL commands to replicate Hive ACID tables between CDP Private Cloud Base clusters, ensure that your source cluster is on CDP Private Cloud Base 7.1.8 or a higher version. [QUESTION] Does Cloudera CDP Base 7.1.7 support the REPL command? [ANSWER] '

In [32]:
torch.manual_seed(42)
print('Tokenizing ...')
input_ids = tokenizer([prompt], return_tensors="pt", truncation=True).to('cuda')
streamer = TextStreamer(tokenizer, skip_prompt=True)
print('Start inference:', str(datetime.now()))
_ = model.generate(
    **input_ids,
    streamer=streamer,
    max_new_tokens=100,
    do_sample=False,
    # top_p=0.9,
    # temperature=0.1
)
print('Completed:', str(datetime.now()))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Tokenizing ...
Start inference: 2024-04-18 15:23:17.673273


No, Cloudera CDP Base 7.1.7 does not support the REPL command for replicating Hive ACID tables between CDP Private Cloud Base clusters. The text above states that the source cluster must be on CDP Private Cloud Base 7.1.8 or a higher version to use the REPL command.</s>
Completed: 2024-04-18 15:23:23.012936
